In [2]:
from tqdm import tqdm
import glob
import joblib
import librosa
import numpy as np
import os
import pandas as pd

In [4]:
lst = []

#emo_dict = {'angry':0, 'disgust':1, 'fearful':2, 'happy':3, 'sad':4, 'surprised':5, 'neutral':6 ,'calm':7}
emo_dict = {'angry':0, 'fearful':1, 'happy':2, 'sad':3, 'calm':4}
#ravdess_files = glob.glob(os.path.join('/home/datahack/Documents/RAVDESS_DEST','*','*.wav'))
#ravdess_files = glob.glob(os.path.join('/home/datahack/Documents/VOICE_DATASET','*','*.wav'))
ravdess_files = glob.glob(os.path.join('/home/datahack/catkin_ws/src/rz_se/src/data/train','*','*.wav'))
for wav in tqdm(ravdess_files):
    #label_string = os.path.basename(wav).split('.')[0].split('_')[1]
    label_string = wav.split("/")[-2]
    #label = emo_dict[label_string]
    label = int(label_string)

    try:
        X, sample_rate = librosa.load(wav, res_type='kaiser_best', sr=48000)
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 

        arr = mfccs, label
        lst.append(arr)

    except ValueError:
        continue
        

100%|██████████| 285/285 [00:06<00:00, 44.00it/s]


In [5]:
# Creating X and y: zip makes a list of all the first elements, and a list of all the second elements.
X, y = zip(*lst)

In [6]:
import numpy as np
X = np.asarray(X)
y = np.asarray(y)


X.shape, y.shape

((285, 40), (285,))

In [75]:
type(X[0])

numpy.float32

In [7]:
X_name = 'X.joblib'
y_name = 'y.joblib'
save_dir = '/home/datahack/Documents/RAVDESS_model'

savedX = joblib.dump(X, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [10]:
x_traincnn.shape

(204, 40, 1)

In [11]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [13]:
model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(40,1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(5))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 40, 128)           768       
_________________________________________________________________
activation_4 (Activation)    (None, 40, 128)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 5, 128)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5, 128)            82048     
_________________________________________________________________
activation_5 (Activation)    (None, 5, 128)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 5, 128)            0         
__________

In [15]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [16]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=16, epochs=1000, validation_data=(x_testcnn, y_test))

Train on 204 samples, validate on 101 samples
Epoch 1/1000
204/204 [==============================] - 1s 6ms/step - loss: 7.3103 - acc: 0.1863 - val_loss: 1.6617 - val_acc: 0.1980
Epoch 2/1000
204/204 [==============================] - 0s 216us/step - loss: 4.0703 - acc: 0.2451 - val_loss: 1.6178 - val_acc: 0.4554
Epoch 3/1000
204/204 [==============================] - 0s 225us/step - loss: 3.6943 - acc: 0.2206 - val_loss: 1.8756 - val_acc: 0.2475
Epoch 4/1000
204/204 [==============================] - 0s 204us/step - loss: 3.8799 - acc: 0.2108 - val_loss: 2.2813 - val_acc: 0.1980
Epoch 5/1000
204/204 [==============================] - 0s 202us/step - loss: 3.9442 - acc: 0.2598 - val_loss: 2.0801 - val_acc: 0.3168
Epoch 6/1000
204/204 [==============================] - 0s 238us/step - loss: 3.5992 - acc: 0.2843 - val_loss: 1.4830 - val_acc: 0.2673
Epoch 7/1000
204/204 [==============================] - 0s 225us/step - loss: 3.7016 - acc: 0.2500 - val_loss: 1.9654 - val_acc: 0.2871
Epoc

Epoch 120/1000
204/204 [==============================] - 0s 195us/step - loss: 0.8198 - acc: 0.7206 - val_loss: 0.7339 - val_acc: 0.7921
Epoch 121/1000
204/204 [==============================] - 0s 187us/step - loss: 0.8095 - acc: 0.6912 - val_loss: 0.7380 - val_acc: 0.7822
Epoch 122/1000
204/204 [==============================] - 0s 218us/step - loss: 0.8765 - acc: 0.6471 - val_loss: 0.7277 - val_acc: 0.7921
Epoch 123/1000
204/204 [==============================] - 0s 190us/step - loss: 0.8171 - acc: 0.6814 - val_loss: 0.8433 - val_acc: 0.6733
Epoch 124/1000
204/204 [==============================] - 0s 179us/step - loss: 0.7798 - acc: 0.7206 - val_loss: 0.7347 - val_acc: 0.7822
Epoch 125/1000
204/204 [==============================] - 0s 193us/step - loss: 0.8562 - acc: 0.6912 - val_loss: 0.7254 - val_acc: 0.7525
Epoch 126/1000
204/204 [==============================] - 0s 183us/step - loss: 0.8041 - acc: 0.6863 - val_loss: 0.6936 - val_acc: 0.8218
Epoch 127/1000
204/204 [==========

204/204 [==============================] - 0s 215us/step - loss: 0.5377 - acc: 0.8186 - val_loss: 0.4711 - val_acc: 0.8614
Epoch 239/1000
204/204 [==============================] - 0s 209us/step - loss: 0.4081 - acc: 0.8971 - val_loss: 0.4715 - val_acc: 0.8218
Epoch 240/1000
204/204 [==============================] - 0s 206us/step - loss: 0.4745 - acc: 0.8333 - val_loss: 0.4529 - val_acc: 0.8515
Epoch 241/1000
204/204 [==============================] - 0s 226us/step - loss: 0.4605 - acc: 0.8382 - val_loss: 0.4847 - val_acc: 0.8515
Epoch 242/1000
204/204 [==============================] - 0s 204us/step - loss: 0.4622 - acc: 0.8284 - val_loss: 0.4435 - val_acc: 0.8713
Epoch 243/1000
204/204 [==============================] - 0s 195us/step - loss: 0.4495 - acc: 0.8627 - val_loss: 0.4859 - val_acc: 0.8416
Epoch 244/1000
204/204 [==============================] - 0s 233us/step - loss: 0.4580 - acc: 0.8627 - val_loss: 0.5233 - val_acc: 0.8218
Epoch 245/1000
204/204 [=========================

204/204 [==============================] - 0s 199us/step - loss: 0.3018 - acc: 0.8971 - val_loss: 0.4148 - val_acc: 0.8614
Epoch 357/1000
204/204 [==============================] - 0s 191us/step - loss: 0.3141 - acc: 0.9069 - val_loss: 0.3409 - val_acc: 0.8713
Epoch 358/1000
204/204 [==============================] - 0s 187us/step - loss: 0.3081 - acc: 0.9118 - val_loss: 0.3543 - val_acc: 0.8614
Epoch 359/1000
204/204 [==============================] - 0s 187us/step - loss: 0.2911 - acc: 0.8873 - val_loss: 0.3630 - val_acc: 0.8713
Epoch 360/1000
204/204 [==============================] - 0s 196us/step - loss: 0.2956 - acc: 0.9020 - val_loss: 0.3449 - val_acc: 0.8713
Epoch 361/1000
204/204 [==============================] - 0s 208us/step - loss: 0.2810 - acc: 0.9265 - val_loss: 0.3954 - val_acc: 0.8515
Epoch 362/1000
204/204 [==============================] - 0s 197us/step - loss: 0.3298 - acc: 0.8873 - val_loss: 0.3184 - val_acc: 0.8911
Epoch 363/1000
204/204 [=========================

204/204 [==============================] - 0s 217us/step - loss: 0.2014 - acc: 0.9314 - val_loss: 0.2677 - val_acc: 0.9010
Epoch 475/1000
204/204 [==============================] - 0s 183us/step - loss: 0.1507 - acc: 0.9706 - val_loss: 0.2993 - val_acc: 0.8812
Epoch 476/1000
204/204 [==============================] - 0s 184us/step - loss: 0.2056 - acc: 0.9265 - val_loss: 0.2582 - val_acc: 0.8614
Epoch 477/1000
204/204 [==============================] - 0s 183us/step - loss: 0.2290 - acc: 0.9216 - val_loss: 0.2672 - val_acc: 0.8614
Epoch 478/1000
204/204 [==============================] - 0s 205us/step - loss: 0.2418 - acc: 0.9314 - val_loss: 0.2908 - val_acc: 0.8713
Epoch 479/1000
204/204 [==============================] - 0s 188us/step - loss: 0.1719 - acc: 0.9559 - val_loss: 0.3152 - val_acc: 0.8713
Epoch 480/1000
204/204 [==============================] - 0s 188us/step - loss: 0.2151 - acc: 0.9363 - val_loss: 0.3032 - val_acc: 0.8614
Epoch 481/1000
204/204 [=========================

204/204 [==============================] - 0s 211us/step - loss: 0.1182 - acc: 0.9608 - val_loss: 0.2477 - val_acc: 0.8911
Epoch 593/1000
204/204 [==============================] - 0s 194us/step - loss: 0.1476 - acc: 0.9559 - val_loss: 0.2560 - val_acc: 0.9109
Epoch 594/1000
204/204 [==============================] - 0s 194us/step - loss: 0.1489 - acc: 0.9510 - val_loss: 0.3148 - val_acc: 0.8812
Epoch 595/1000
204/204 [==============================] - 0s 188us/step - loss: 0.1274 - acc: 0.9657 - val_loss: 0.2435 - val_acc: 0.8812
Epoch 596/1000
204/204 [==============================] - 0s 186us/step - loss: 0.1557 - acc: 0.9510 - val_loss: 0.2552 - val_acc: 0.8812
Epoch 597/1000
204/204 [==============================] - 0s 199us/step - loss: 0.1270 - acc: 0.9510 - val_loss: 0.2811 - val_acc: 0.9010
Epoch 598/1000
204/204 [==============================] - 0s 197us/step - loss: 0.1343 - acc: 0.9657 - val_loss: 0.2548 - val_acc: 0.9010
Epoch 599/1000
204/204 [=========================

204/204 [==============================] - 0s 189us/step - loss: 0.1018 - acc: 0.9706 - val_loss: 0.2506 - val_acc: 0.8911
Epoch 711/1000
204/204 [==============================] - 0s 217us/step - loss: 0.1111 - acc: 0.9559 - val_loss: 0.2241 - val_acc: 0.8812
Epoch 712/1000
204/204 [==============================] - 0s 193us/step - loss: 0.1362 - acc: 0.9559 - val_loss: 0.2399 - val_acc: 0.9109
Epoch 713/1000
204/204 [==============================] - 0s 188us/step - loss: 0.0864 - acc: 0.9804 - val_loss: 0.2289 - val_acc: 0.9010
Epoch 714/1000
204/204 [==============================] - 0s 190us/step - loss: 0.0873 - acc: 0.9755 - val_loss: 0.3021 - val_acc: 0.8812
Epoch 715/1000
204/204 [==============================] - 0s 197us/step - loss: 0.1258 - acc: 0.9608 - val_loss: 0.2253 - val_acc: 0.8812
Epoch 716/1000
204/204 [==============================] - 0s 211us/step - loss: 0.0882 - acc: 0.9853 - val_loss: 0.2626 - val_acc: 0.8812
Epoch 717/1000
204/204 [=========================

204/204 [==============================] - 0s 192us/step - loss: 0.0541 - acc: 0.9902 - val_loss: 0.2322 - val_acc: 0.9010
Epoch 829/1000
204/204 [==============================] - 0s 201us/step - loss: 0.0701 - acc: 0.9902 - val_loss: 0.2791 - val_acc: 0.8911
Epoch 830/1000
204/204 [==============================] - 0s 189us/step - loss: 0.0705 - acc: 0.9853 - val_loss: 0.2518 - val_acc: 0.9010
Epoch 831/1000
204/204 [==============================] - 0s 185us/step - loss: 0.0836 - acc: 0.9755 - val_loss: 0.2594 - val_acc: 0.8812
Epoch 832/1000
204/204 [==============================] - 0s 185us/step - loss: 0.0813 - acc: 0.9706 - val_loss: 0.2617 - val_acc: 0.9109
Epoch 833/1000
204/204 [==============================] - 0s 190us/step - loss: 0.0783 - acc: 0.9755 - val_loss: 0.2312 - val_acc: 0.9010
Epoch 834/1000
204/204 [==============================] - 0s 220us/step - loss: 0.0756 - acc: 0.9804 - val_loss: 0.2265 - val_acc: 0.9010
Epoch 835/1000
204/204 [=========================

204/204 [==============================] - 0s 211us/step - loss: 0.0460 - acc: 0.9951 - val_loss: 0.2458 - val_acc: 0.8911
Epoch 947/1000
204/204 [==============================] - 0s 194us/step - loss: 0.0500 - acc: 0.9853 - val_loss: 0.2406 - val_acc: 0.9010
Epoch 948/1000
204/204 [==============================] - 0s 185us/step - loss: 0.0643 - acc: 0.9853 - val_loss: 0.2435 - val_acc: 0.9010
Epoch 949/1000
204/204 [==============================] - 0s 193us/step - loss: 0.0386 - acc: 0.9902 - val_loss: 0.2361 - val_acc: 0.8812
Epoch 950/1000
204/204 [==============================] - 0s 207us/step - loss: 0.0611 - acc: 0.9853 - val_loss: 0.2553 - val_acc: 0.9010
Epoch 951/1000
204/204 [==============================] - 0s 196us/step - loss: 0.0445 - acc: 0.9951 - val_loss: 0.2526 - val_acc: 0.8911
Epoch 952/1000
204/204 [==============================] - 0s 186us/step - loss: 0.0619 - acc: 0.9804 - val_loss: 0.2441 - val_acc: 0.8911
Epoch 953/1000
204/204 [=========================

### Predicción

In [65]:
#wav_path = '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/male_angry.wav'
#wav_path = '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/male_sad.wav'
#wav_path = '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/male_fearful.wav'
#wav_path = '/home/datahack/catkin_ws/src/mic2wav/src/wav_files/female_calm.wav'
#
#wav_path = '/home/datahack/Documents/RAVDESS_DEST/Audio_Speech_Actors_01-24/Speech_surprised_0051.wav'
wav_path = '/home/datahack/Documents/VOICE_DATASET/enfado3.wav'

In [66]:
X, sample_rate = librosa.load(wav_path, res_type='kaiser_best')
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 

In [67]:
mfccs_test = np.expand_dims(mfccs.reshape(-1,1), 0)

In [68]:
mfccs_test.shape

(1, 40, 1)

In [69]:
model.predict_proba(mfccs_test)

array([[0.90461683, 0.07215531, 0.00588792, 0.0036193 , 0.01372064]],
      dtype=float32)

In [70]:
model.predict_classes(mfccs_test)

array([0])

### Predicción 

In [104]:
from pysndfx import AudioEffectsChain
import math
import python_speech_features
import scipy as sp

In [105]:
def reduce_noise_centroid_mb(y, sr):

    cent = librosa.feature.spectral_centroid(y=y, sr=sr)

    threshold_h = np.max(cent)
    threshold_l = np.min(cent)

    less_noise = AudioEffectsChain().lowshelf(gain=-30.0, frequency=threshold_l, slope=0.5).highshelf(gain=-30.0, frequency=threshold_h, slope=0.5).limiter(gain=10.0)
    # less_noise = AudioEffectsChain().lowpass(frequency=threshold_h).highpass(frequency=threshold_l)
    y_cleaned = less_noise(y)


    cent_cleaned = librosa.feature.spectral_centroid(y=y_cleaned, sr=sr)
    columns, rows = cent_cleaned.shape
    boost_h = math.floor(rows/3*2)
    boost_l = math.floor(rows/6)
    boost = math.floor(rows/3)

    # boost_bass = AudioEffectsChain().lowshelf(gain=20.0, frequency=boost, slope=0.8)
    boost_bass = AudioEffectsChain().lowshelf(gain=16.0, frequency=boost_h, slope=0.5)#.lowshelf(gain=-20.0, frequency=boost_l, slope=0.8)
    y_clean_boosted = boost_bass(y_cleaned)

    return y_clean_boosted

In [106]:
wav_path = '/home/datahack/Documents/NOISE/raw_1553100814.28.wav'

In [107]:
X, sample_rate = librosa.load(wav_path, res_type='kaiser_best')

In [108]:
mfccs = np.mean(librosa.feature.mfcc(y=reduce_noise_centroid_mb(X, sample_rate), sr=sample_rate, n_mfcc=40).T,axis=0) 

In [109]:
mfccs_test = np.expand_dims(mfccs.reshape(-1,1), 0)

In [110]:
mfccs_test.shape

(1, 40, 1)

In [111]:
model.predict_classes(mfccs_test)

array([0])

In [112]:
mfccs.shape

(40,)

In [113]:
librosa.feature.mfcc(y=reduce_noise_centroid_mb(X, sample_rate), sr=sample_rate, n_mfcc=40).shape

(40, 118)